
# 🌲 Recursive Neural Network (RecNN) for Sentiment Classification

This notebook demonstrates how to simulate a **Recursive Neural Network (RecNN)** in PyTorch for structured input, such as parse trees. The task is to classify sentiment using a simplified binary tree structure.

---


In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F


In [ ]:

# Define a binary tree node
class TreeNode:
    def __init__(self, left=None, right=None, value=None):
        self.left = left
        self.right = right
        self.value = value  # Word index or embedding input


In [ ]:

# Define Recursive Neural Network
class RecNN(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(RecNN, self).__init__()
        self.W = nn.Linear(2 * hidden_dim, hidden_dim)
        self.embedding = nn.Embedding(10000, input_dim)
        self.projection = nn.Linear(input_dim, hidden_dim)

    def forward(self, node):
        if node.left is None and node.right is None:
            embed = self.embedding(torch.tensor([node.value]))
            return self.projection(embed).squeeze(0)
        else:
            left_vec = self.forward(node.left)
            right_vec = self.forward(node.right)
            combined = torch.cat((left_vec, right_vec), dim=-1)
            return F.relu(self.W(combined))


In [ ]:

# Simulate a binary tree: (great (not movie))
left_leaf = TreeNode(value=2)   # "not"
right_leaf = TreeNode(value=3)  # "movie"
intermediate = TreeNode(left=left_leaf, right=right_leaf)
root = TreeNode(left=TreeNode(value=1), right=intermediate)  # "great"

# Create the model and get sentence vector
model = RecNN(input_dim=50, hidden_dim=64)
sentence_vector = model(root)
print("Final representation vector:", sentence_vector)
